In [1]:
import os
import phoenix as px

# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://192.168.1.122:6006"

client = px.Client()
dataset = client.get_dataset(
    name="my-chinese-ods-dataset",
    version_id="RGF0YXNldFZlcnNpb246OQ=="
)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.12.0) and client (8.27.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


In [2]:
import phoenix
print(phoenix.__version__)

8.27.1


In [3]:
print(dataset)

Dataset(id='RGF0YXNldDo3', version_id='RGF0YXNldFZlcnNpb246OQ==')


In [4]:
print(dir(dataset))

['__annotations__', '__class__', '__dataclass_fields__', '__dataclass_params__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__ne__', '__new__', '__post_init__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_keys', 'as_dataframe', 'examples', 'from_dict', 'id', 'version_id']


In [5]:
print(dataset.as_dataframe())

                                                 input  \
example_id                                               
RGF0YXNldEV4YW1wbGU6NTA=      {'input': '艾菲爾鐵塔位於哪裡？ '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'input': '中國的長城有多長？ '}   
RGF0YXNldEV4YW1wbGU6NTI=     {'input': '最大的熱帶雨林是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTM=    {'input': '地球上最高的山是哪一座？ '}   
RGF0YXNldEV4YW1wbGU6NTQ=    {'input': '世界上最長的河流是什麼？ '}   
RGF0YXNldEV4YW1wbGU6NTU=       {'input': '蒙娜麗莎是誰畫的？ '}   
RGF0YXNldEV4YW1wbGU6NTY=        {'input': '人體有幾塊骨頭？ '}   
RGF0YXNldEV4YW1wbGU6NTc=  {'input': '太陽系中最大的行星是哪一顆？ '}   
RGF0YXNldEV4YW1wbGU6NTg=  {'input': '《羅密歐與茱麗葉》是誰寫的？ '}   
RGF0YXNldEV4YW1wbGU6NTk=  {'input': '第一次登月是什麼時候發生的？ '}   

                                                                     output  \
example_id                                                                    
RGF0YXNldEV4YW1wbGU6NTA=                        {'output': '艾菲爾鐵塔位於法國巴黎。 '}   
RGF0YXNldEV4YW1wbGU6NTE=       {'output': '中國的長城長約 13,171 英里（21,19

In [6]:
df = dataset.as_dataframe()
print(df.columns)

Index(['input', 'output', 'metadata'], dtype='object')


In [7]:
df["input"] = df["input"].apply(lambda x: x['input'])
df["output"] = df["output"].apply(lambda x: x['output'])


In [8]:
MY_CUSTOM_TEMPLATE = '''
    You are evaluating whether the output mentions any person's name, such as common first names, full names, or references to individuals.
    [BEGIN DATA]
    ************
    [Question]: {input}
    ************
    [Response]: {output}
    [END DATA]


    Please focus on the tone of the response.
    Your answer must be single word, either "yes" or "no"
    '''

In [9]:
import openai
import os
from phoenix.evals import llm_classify, OpenAIModel

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

res = client.chat.completions.create(
model="o1",
temperature=0.5,
messages=[
{"role": "system", "content": "RAG系統的output評估器"},
{"role": "user", "content": "RAG系統output:《蒙娜麗莎》是義大利文藝復興時期藝術家李奧納多·達文西的作品，請告訴我這個回應中有無出現人名，你只能回答有或沒有"},
]
)
print(res.choices[0].message.content)


有


In [10]:
import openai
import os
from phoenix.evals import llm_classify, OpenAIModel

rails = ['yes', 'no']

model = OpenAIModel(
    model="o1",
    temperature=0.5,
    api_key=os.getenv("OPENAI_API_KEY"),     # 你的API Key
    base_url=os.getenv("OPENAI_API_BASE")    # 你的 Base URL
)

# 在外面跑 llm_classify（批次對整個df做分類）
positive_eval = llm_classify(
    data=df,
    template=MY_CUSTOM_TEMPLATE,
    model=model,
    rails=rails,
    concurrency=5
)


🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


llm_classify |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

In [14]:
print(positive_eval.label)

example_id
RGF0YXNldEV4YW1wbGU6NTA=     no
RGF0YXNldEV4YW1wbGU6NTE=     no
RGF0YXNldEV4YW1wbGU6NTI=     no
RGF0YXNldEV4YW1wbGU6NTM=     no
RGF0YXNldEV4YW1wbGU6NTQ=     no
RGF0YXNldEV4YW1wbGU6NTU=    yes
RGF0YXNldEV4YW1wbGU6NTY=     no
RGF0YXNldEV4YW1wbGU6NTc=     no
RGF0YXNldEV4YW1wbGU6NTg=    yes
RGF0YXNldEV4YW1wbGU6NTk=     no
Name: label, dtype: object


In [15]:
labels_mapping = positive_eval["label"].to_dict()
print(labels_mapping)

{'RGF0YXNldEV4YW1wbGU6NTA=': 'no', 'RGF0YXNldEV4YW1wbGU6NTE=': 'no', 'RGF0YXNldEV4YW1wbGU6NTI=': 'no', 'RGF0YXNldEV4YW1wbGU6NTM=': 'no', 'RGF0YXNldEV4YW1wbGU6NTQ=': 'no', 'RGF0YXNldEV4YW1wbGU6NTU=': 'yes', 'RGF0YXNldEV4YW1wbGU6NTY=': 'no', 'RGF0YXNldEV4YW1wbGU6NTc=': 'no', 'RGF0YXNldEV4YW1wbGU6NTg=': 'yes', 'RGF0YXNldEV4YW1wbGU6NTk=': 'no'}


In [58]:
from phoenix.experiments.types import Example

def my_task(example: Example) -> str:
    return example
    # return {"a": 1, "b": 2}

In [56]:
client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

# Define an evaluator. This just an example.
def exact_match(input, output) -> str:
    #return 1.0 if output is f"Hello {input}" else 1.0
    # print(input)
    # print(output)    
    output_message = output["output"]["output"]
    print(output_message)    
    
    MY_CUSTOM_TEMPLATE = f'''
You are evaluating whether the output mentions any person's name, such as common first names, full names, or references to individuals.
[BEGIN DATA]
[Response]: {output_message}
[END DATA]


Please focus on the tone of the response.
Your answer must be single word, either "yes" or "no"
'''
    
    # print(MY_CUSTOM_TEMPLATE)
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    
    content = res.choices[0].message.content
    print(content)
    content = content.strip().lower() 
    
    return content
    # return "yes" if res.choices[0].message.content == "yes" else "no" 
    
    
def hallucination_test(input, output) -> str:
    print(input)
    # print(output)    
    input_message = input["input"]
    print(input_message)
    
    output_message = output["output"]["output"]
    print(output_message)    
    
    MY_CUSTOM_TEMPLATE = f'''

[BEGIN DATA]
[Question]: {input_message}
[Response]: {output_message}
[END DATA]

請判斷[Response]回答[Question]的內容中，是否有與事實不符的地方。
如果有，則回答 "幻覺"；沒有幻覺的話，則回答 "事實"。

不要其他說明，必須回答單詞即可。
'''
    
    # print(MY_CUSTOM_TEMPLATE)
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    
    content = res.choices[0].message.content
    print(content)
    content = content.strip().lower() 
    
    return content
    # return "yes" if res.choices[0].message.content == "yes" else "no" 
    
    
# Store the evaluators for later use
evaluators = [exact_match, hallucination_test]

In [59]:
# Run an experiment
from phoenix.experiments import run_experiment

experiment = run_experiment(dataset, my_task, evaluators=evaluators)

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.12.0) and client (8.27.1) versions are mismatched and may have compatibility issues.
  warnings.warn(
🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


🧪 Experiment started.
📺 View dataset experiments: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/experiments
🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo3/compare?experimentId=RXhwZXJpbWVudDozMQ==


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/20 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:60: UserWarning: The Phoenix server (8.12.0) and client (8.27.1) versions are mismatched and may have compatibility issues.
  warnings.warn(


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/phoenix/experiments/functions.py", line 503, in sync_evaluate_run
    result = evaluator.evaluate(
  File "/opt/conda/lib/python3.10/site-packages/phoenix/experiments/evaluators/utils.py", line 215, in evaluate
    result = func(*bound_signature.args, **bound_signature.kwargs)
  File "/tmp/ipykernel_57/1245691075.py", line 11, in exact_match
    output_message = output["output"]["output"]
KeyError: 'output'

The above exception was the direct cause of the following exception:

RuntimeError: evaluator failed for example id 'RGF0YXNldEV4YW1wbGU6NTA=', repetition 1

Retries exhausted after 1 attempts: Client error '422 Unprocessable Entity' for url 'http://192.168.1.122:6006/v1/experiment_evaluations'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422
{'input': '艾菲爾鐵塔位於哪裡？ '}
艾菲爾鐵塔位於哪裡？ 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/